In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from numpy.random import seed

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def acc_f1(y_true, y_pred, abnormal = 0):
    cm = np.zeros((2,2))
    cm[0][0] = np.sum((y_true != abnormal)&(y_pred != abnormal))
    cm[0][1] = np.sum((y_true != abnormal)&(y_pred == abnormal))
    cm[1][0] = np.sum((y_true == abnormal)&(y_pred != abnormal))
    cm[1][1] = np.sum((y_true == abnormal)&(y_pred == abnormal))
    precision = cm[1][1] / np.sum(cm[:,1])
    recall = cm[1][1] / np.sum(cm[1,:])
    f1 = 2*precision*recall/(precision+recall)
    return (cm[0][0] + cm[1][1])/(np.sum(cm)), f1
def IOU(y_true, y_pred, abnormal = 0):
    U = np.sum((y_true == abnormal)|(y_pred == abnormal))
    I = np.sum((y_true == abnormal)&(y_pred == abnormal))
    return I/U

# data loading 

In [32]:
train.shape, test_a.shape, ab_a.shape

((1043638, 111), (176071, 111), (33374, 111))

In [33]:
def del_f(i):
    del train[i]
    del test_a[i]
    del ab_a[i]

In [34]:
del_f(14) # delete meaningless data 1 : sensor_smk

In [35]:
del_f(20) # delete meaningless data 1 : sensor_air

In [36]:
del_f(65) # delete meaningless data 1 : sensor_cycle

In [37]:
def nor(data2):
    data_norm2 = (data2 - data2.min()) / (data2.max() - data2.min())
    data_norm2 = data_norm2.fillna(0)
    X_data2 = []
    for i in range(0, data_norm2.shape[0], strides):
        #if (len(X_data2) % 200==0):
        #    print(len(X_data2))
        temp = data_norm2.iloc[i:i+interval].values
        if temp.shape[0] != interval:
            continue
        X_data2.append(np.expand_dims(temp, axis=0))
    X_data2 = np.concatenate(X_data2, axis=0)
    print(X_data2.shape)
    return X_data2

In [38]:
interval = 10
strides = 2

In [39]:
train_a=nor(train)
test_b=nor(test_a)
ab_b=nor(ab_a)

(521815, 10, 108)
(88031, 10, 108)
(16683, 10, 108)


In [40]:
train_n= (np.reshape(train_a, [train_a.shape[0], -1]))
test_n= (np.reshape(test_b, [test_b.shape[0], -1]))
ab_n= (np.reshape(ab_b, [ab_b.shape[0], -1]))

In [41]:
train_n.shape, test_n.shape, ab_n.shape

((521815, 1080), (88031, 1080), (16683, 1080))

# 1. Auto encoder for SVM

In [42]:
### Define the encoder dimension
encoding_dim = 32

In [43]:
# define the number of features
ncol = train_n.shape[1]

In [66]:
input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(512, activation = 'relu')(input_dim)
encoded2 = Dense(128, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded2)
    
decoded1 = Dense(128, activation = 'relu')(encoded3)
decoded2 = Dense(512, activation = 'relu')(decoded1)
decoded3 = Dense(ncol, activation = 'sigmoid')(decoded2)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded3)
# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mse')

In [67]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1080)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               553472    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_11 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_12 (Dense)             (None, 1080)              554040    
Total para

In [69]:
X_train, X_test = train_test_split(train_n, train_size = 0.9, random_state = seed(333))

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [70]:
X_train.shape, X_test.shape

((469633, 1080), (52182, 1080))

In [71]:
autoencoder.fit(X_train, X_train, nb_epoch = 50, batch_size =1000, shuffle = False, validation_data = (X_test, X_test))

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 469633 samples, validate on 52182 samples
Epoch 1/50
469633/469633 [==============================] - 10s 22us/step - loss: 0.0067 - val_loss: 0.0012
Epoch 2/50
469633/469633 [==============================] - 10s 21us/step - loss: 7.6078e-04 - val_loss: 5.1403e-04
Epoch 3/50
469633/469633 [==============================] - 10s 21us/step - loss: 4.9076e-04 - val_loss: 4.3547e-04
Epoch 4/50
469633/469633 [==============================] - 10s 21us/step - loss: 4.1720e-04 - val_loss: 3.7065e-04
Epoch 5/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.5568e-04 - val_loss: 3.2116e-04
Epoch 6/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.2296e-04 - val_loss: 3.0720e-04
Epoch 7/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.0084e-04 - val_loss: 2.8076e-04
Epoch 8/50
469633/469633 [==============================] - 10s 21us/step - loss: 2.7437e-04 - val_loss: 2.5093e-04
Epoch 9/50
469633/469633 [===

In [72]:
encoder = Model(inputs = input_dim, outputs = encoded3)
encoded_input = Input(shape = (encoding_dim, ))

In [73]:
encoded_train = pd.DataFrame(encoder.predict(train_n))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(test_n))
encoded_test = encoded_test.add_prefix('feature_')

encoded_ab = pd.DataFrame(encoder.predict(ab_n))
encoded_ab = encoded_ab.add_prefix('feature_')

# one class svm

In [84]:
from sklearn.preprocessing import minmax_scale


In [85]:
train_scaled = minmax_scale(encoded_train, axis = 0)
test_scaled = minmax_scale(encoded_test, axis = 0)
ab_scaled = minmax_scale(encoded_ab, axis = 0)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  if __name__ == '__main__':
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  from ipykernel import kernelapp as app
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  app.launch_new_instance()


In [86]:
    p_svm = OneClassSVM(max_iter=1000, verbose=True, nu=0.02, kernel = 'rbf', gamma=0.0001)
    p_svm.fit(train_scaled)
    p_svm_test = p_svm.predict(test_scaled)
    set(p_svm_test)
    p_svm_abnor = p_svm.predict(ab_scaled)
    set(p_svm_abnor)
    p_pred = np.concatenate([p_svm_test, p_svm_abnor], axis=0)
    p_Y = np.array([1] * test_scaled.shape[0] + [-1] * ab_scaled.shape[0])
    p_result = acc_f1(p_Y, p_pred, abnormal = -1)
    acc_f1(p_Y, p_pred, abnormal = -1)
    result_IOU = IOU(p_Y, p_pred, abnormal = -1)    
    print("Accuracy: %.8f"%((p_result[1]+p_result[0])/2))
    print("IOU: %.8f"%(result_IOU))

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.21708498
IOU: 0.15931967


# LSTM encoder for Isolation forest

In [121]:
n_dimensions=32
timesteps=10
input_dim=train_a.shape[2]

In [123]:
def get_model(x):
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(n_dimensions, return_sequences=False, name="encoder")(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True, name='decoder')(decoded)

    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    return autoencoder, encoder

get_model(train_a)


(<keras.engine.training.Model at 0x7f70583c93c8>,
 <keras.engine.training.Model at 0x7f705836b0b8>)

In [125]:
autoencoder, encoder = get_model(n_dimensions)
autoencoder.compile(optimizer='rmsprop', loss='mse', 
                    metrics=['acc', 'cosine_proximity'])

history = autoencoder.fit(train_a, train_a, batch_size=500, epochs=10)


Epoch 1/10
521815/521815 [==============================] - 44s 84us/step - loss: 0.0050 - acc: 0.2065 - cosine_proximity: -0.9883
Epoch 2/10
521815/521815 [==============================] - 41s 79us/step - loss: 0.0012 - acc: 0.2499 - cosine_proximity: -0.9980
Epoch 3/10
521815/521815 [==============================] - 41s 78us/step - loss: 0.0012 - acc: 0.2811 - cosine_proximity: -0.9982
Epoch 4/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3115 - cosine_proximity: -0.9983
Epoch 5/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3241 - cosine_proximity: -0.9983
Epoch 6/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3255 - cosine_proximity: -0.9984
Epoch 7/10
521815/521815 [==============================] - 41s 79us/step - loss: 0.0011 - acc: 0.3228 - cosine_proximity: -0.9984
Epoch 8/10
521815/521815 [==============================] - 41s 78us/step - loss: 0

In [132]:
encoded_train = encoder.predict(train_a)
encoded_test = encoder.predict(test_b)
encoded_ab = encoder.predict(ab_b)

In [133]:
encoded_train.shape, encoded_test.shape, encoded_ab.shape

((521815, 32), (88031, 32), (16683, 32))

In [147]:
clf3=IsolationForest(contamination=0.01, random_state=333, n_jobs=-1, behaviour= "new", max_features=32, max_samples=200)
clf3.fit(encoded_train)
pred_test3 = clf3.predict(encoded_test)
pred_abnor3= clf3.predict(encoded_ab)

pred_if3 = np.concatenate([pred_test3, pred_abnor3], axis=0)
Y_data_if3 = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
result_if3 = acc_f1(Y_data_if3, pred_if3, abnormal = -1)
#acc_f1(Y_data_if3, pred_if3, abnormal = -1)
result_IOU = IOU(Y_data_if3, pred_if3, abnormal = -1)

print("Accuracy: %.8f"%((result_if3[0]+result_if3[1])/2))
print("IOU: %.8f"%(result_IOU))

Accuracy: 0.81449375
IOU: 0.55189096


In [151]:
cm = confusion_matrix(Y_data_if3, pred_if3)
cm

array([[10609,  6074],
       [ 2540, 85491]])

# Grid search for Isolation forest

In [156]:
from sklearn.model_selection import GridSearchCV
scores = ['precision', 'recall']
tuned_parameters = [{'contamination':[ 0.01, 0.02, 0.03], 
                     'max_samples': [25, 30, 35, 40, 45,100],
                    'random_state':[3, 33, 333],
                    'max_features':[32,16]}]

In [157]:
train_y=np.ones((521815, 1)) # every elements is 1 in 521815 by 1 array 

In [ ]:
for score in scores:
    clf = GridSearchCV(IsolationForest(n_jobs=-1, behaviour="new"), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(encoded_train, train_y)
   
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: Undefine

In [159]:
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
       print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
print(clf.best_params_)

0.592 (+/-0.409) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 25, 'max_features': 32}
0.493 (+/-0.012) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 25, 'max_features': 32}
0.792 (+/-0.509) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 25, 'max_features': 32}
0.685 (+/-0.516) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 30, 'max_features': 32}
0.488 (+/-0.032) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 30, 'max_features': 32}
0.781 (+/-0.540) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 30, 'max_features': 32}
0.568 (+/-0.444) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 35, 'max_features': 32}
0.484 (+/-0.034) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 35, 'max_features': 32}
0.566 (+/-0.452) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 35, 'max_features': 32}
0.569 (+/-0.439) for {'contamination': 0.01, 'random_state': 3, 

In [196]:
err =encoded_ab.shape[0]/(encoded_train.shape[0]+encoded_test.shape[0])
err

0.027356086618588956

In [209]:
clf3=IsolationForest(contamination=err, random_state=333, n_jobs=-1, behaviour= "new", max_features=32, max_samples=25)
clf3.fit(encoded_train)
pred_test3 = clf3.predict(encoded_test)
pred_abnor3= clf3.predict(encoded_ab)

pred_if3 = np.concatenate([pred_test3, pred_abnor3], axis=0)
Y_data_if3 = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
result_if3 = acc_f1(Y_data_if3, pred_if3, abnormal = -1)
#acc_f1(Y_data_if3, pred_if3, abnormal = -1)
result_IOU = IOU(Y_data_if3, pred_if3, abnormal = -1)

print("Accuracy: %.8f"%((result_if3[0]+result_if3[1])/2))
print("IOU: %.8f"%(result_IOU))

Accuracy: 0.85452942
IOU: 0.63627625
